# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.126576e+02     1.781461e+00
 * time: 0.09125709533691406
     1     1.101762e+01     1.101698e+00
 * time: 0.955970048904419
     2    -1.200737e+01     1.152970e+00
 * time: 1.1067330837249756
     3    -3.375422e+01     9.825408e-01
 * time: 1.2450380325317383
     4    -4.737838e+01     7.750459e-01
 * time: 1.396320104598999
     5    -5.684902e+01     2.495177e-01
 * time: 1.5416321754455566
     6    -5.976889e+01     1.713398e-01
 * time: 1.6565911769866943
     7    -6.086071e+01     5.005932e-02
 * time: 1.7645909786224365
     8    -6.127470e+01     1.021412e-01
 * time: 1.8810670375823975
     9    -6.156890e+01     3.476022e-02
 * time: 1.996570110321045
    10    -6.178482e+01     2.954601e-02
 * time: 2.105703115463257
    11    -6.195048e+01     1.980584e-02
 * time: 2.221428155899048
    12    -6.201681e+01     1.689520e-02
 * time: 2.329679012298584
    13    -6.208482e+01     1.557727e-02
 * time: 2.444904088973999

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671065
    AtomicLocal         -18.8557667
    AtomicNonlocal      14.8522641
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485366 
    Xc                  -19.3336818

    total               -62.261666461435
